# Uzbekistan Uurbanization
As part of a project (P162929) looking at urbanization in Uzbekistan, we are generating a number of summary numbers looking at urbanization. These include municipal level numbers comparing change between the cities, and intra city changes using higher resolution imagery.

### City-level analysis
1. Puga Index
2. LEI
3. MIT Urban Form (compactness, discontiguity, expandability, polycentricity
4. Nighttime Lights (VIIRS and DMSP)
5. Built-area change

### Sub-city analysis
1. City Scan

In [3]:
import sys, os, inspect, logging, importlib
import rasterio

import pandas as pd
import geopandas as gpd
import networkx as nx
import GOSTnets as gn
import GOSTnets.load_osm as losm
import GOSTnets.calculate_od_raw as calcOD

from shapely.wkt import loads
from shapely.ops import cascaded_union

cmd_folder = os.path.join("/home/public/Code/GOST")
sys.path.insert(0, cmd_folder)

sys.path.append("../../")

import infrasap.market_access as ma
import GOSTRocks.misc as misc
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.osmMisc as osmMisc
import GOSTRocks.Urban.UrbanRaster

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)


In [4]:
ma.generate_network_raster?

In [7]:
# Define and read in input data
national_boundary = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
national_settlements = "/home/public/Data/GLOBAL/Population/global_settlement_points_v1_01.shp"

inputAOI = r"/home/wb411133/data/Country/UZB/UrbanExtents/Urban_Manual_Extents.shp"
UZB_boundary = r"/home/wb411133/data/Country/UZB/Adm0.shp"
UZB_cities = r"/home/wb411133/data/Country/UZB/Cities_and_UTS_pop.shp"
outputFolder = r"/home/wb411133/data/Country/UZB/urban_summaries"

gridded_pop = "/home/public/Data/GLOBAL/Population/ppp_prj_2020_UZB.tif"
wp_urbanExtents = inputAOI.replace(".shp", "_worldPop_300.csv")
wp_densExtents = inputAOI.replace(".shp", "_worldPop_1500.csv")

viirsFolder = '/home/public/Data/GLOBAL/NighttimeLights/VIIRS_VRT'
dmspFolder = '/home/public/Data/GLOBAL/NighttimeLights/DMSP'
ghslVRT = '/home/public/Data/GLOBAL/GHSL/ghsl.vrt'
floodVRT = '/home/public/Data/COUNTRY/UZB/uzbekistan/CombinedFloodData/FU_PU_Combined.VRT'
osmPBF = '/home/public/Data/COUNTRY/UZB/OSM/uzbekistan-latest.osm.pbf'
bufferedOSM_pbf = '/home/wb411133/data/Country/UZB/uzbekistan-latest_100km_buffer.osm.pbf'

m_proj = {'init':'epsg:3857'} #projection to use when metres are necessary

if not os.path.exists(UZB_boundary):
    national_data = gpd.read_file(national_boundary)
    UZB_b = national_data[national_data['ISO3'] == "UZB"]
    UZB_b.to_file(UZB_boundary)
else:
    UZB_b = gpd.read_file(UZB_boundary)


In [3]:
#Read in digitized extents, the points from Ildus' collection 
#    and the gridded population extents, then combine
manual_extents = "/home/wb411133/data/Country/UZB/UrbanExtents/allUrbanSummaries_simplified.shp"
wp_extents =     "/home/wb411133/data/Country/UZB/UrbanExtents/Urban_Manual_Extents_worldPop_300.csv"
cities =         "/home/wb411133/data/Country/UZB/Ildus_Cities_2019_12_19.gpkg"

manual_e = gpd.read_file(manual_extents)
manual_e = manual_e.loc[:,['Name','geometry']]
wp_extents = pd.read_csv(wp_extents)
wb_extents_geom = [loads(x) for x in wp_extents['geometry']]
wp_extents = gpd.GeoDataFrame(wp_extents.drop(['geometry'], axis=1), geometry = wb_extents_geom, crs = manual_e.crs)
cities = gpd.read_file(cities)

# Merge manual extents with Ildus Cities and with WP extents
manual_e['I_Cities'] = ''
manual_e['wp_extents'] = ''
wp_extents['ID'] = [str(x) for x in wp_extents['ID']]
for idx, row in manual_e.iterrows():
    select_cities = cities[cities.intersects(row['geometry'])]
    manual_e.loc[idx,'I_Cities'] = ";".join(select_cities['Adm. Center\nENGLISH'])
    # identify extents from gridded population
    select_extents = wp_extents[wp_extents.intersects(row['geometry'])]
    if select_extents.shape[0] > 1:
        break
    manual_e.loc[idx, 'wp_extents'] = ";".join(select_extents['ID'])
    
inA = manual_e

# Raster Market Access

In [13]:
cities_access = os.path.join(outputFolder, "seconds_to_cities_2.tif")
cities_access_50k = os.path.join(outputFolder, "seconds_to_cities_50k_2.tif")
traversal_raster = os.path.join(outputFolder, "traversal_time.tif")
market_shed_raster = os.path.join(outputFolder, "traversal_time_marketsheds.tif")
uzb_cities = gpd.read_file(UZB_cities)
uzb_cities = uzb_cities.loc[uzb_cities['Lat'] != 0]
big_cities = uzb_cities.loc[uzb_cities['y2019'] >= 50000]
pop_data = rasterio.open(gridded_pop)

In [11]:
uzb_cities.head()

,Name_UZB,Name_ENG,Lat,Long,Region,Type,Adm_center,y2010,y2015,y2019,c20102019,c20102015,c20152019,geometry
0,Тошкент шаҳар,Tashkent,41.311139,69.279750,Tashkent,city,0.0,2227502.0,2371269.0,2509969.0,112.680886,106.454181,105.849189,POINT (69.27975000000001 41.311139)
1,Нукус ш.,Nukus,42.464722,59.602222,Karakalpakstan,city,2.0,268798.0,298256.0,312385.0,116.215522,110.959159,104.737206,POINT (59.602222 42.464722)
2,Каратау шаҳарча,Karatau,42.089059,60.281151,Karakalpakstan,uts,0.0,2652.0,2402.0,2543.0,95.889894,90.573152,105.870108,POINT (60.281151 42.089059)
3,Манғит ш,Mangit,42.116115,60.061537,Karakalpakstan,city,1.0,35159.0,34060.0,36519.0,103.868142,96.874200,107.219612,POINT (60.061537 42.116115)
4,"""Жумуртов"" шаҳарча",Jumurtov,42.060298,60.238327,Karakalpakstan,uts,0.0,3764.0,3366.0,3621.0,96.200850,89.426142,107.575758,POINT (60.238327 42.060298)


In [5]:
if not os.path.exists(traversal_raster):
    G_loader = losm.OSM_to_network(bufferedOSM_pbf)
    G_loader.generateRoadsGDF()
    roads = G_loader.roadsGPD 
    roads['geometry'] = roads['Wkt']
    roads['speed'] = roads['infra_type'].map(ma.speed_dict)
    roads = roads.sort_values(['speed'])
    roads = roads[~roads['speed'].isnull()]
    traversal_time = ma.generate_network_raster(pop_data, roads)
    traversal_time = traversal_time.astype(pop_data.meta['dtype'])
    with rasterio.open(traversal_raster, 'w', **pop_data.meta) as out:
        out.write_band(1, traversal_time)
else:
    network_r = rasterio.open(traversal_raster)
    traversal_time = network_r.read()[0,:,:]

In [9]:
import skimage.graph as graph
mcp = graph.MCP_Geometric(traversal_time)

In [14]:
importlib.reload(ma)
res = ma.generate_market_sheds(network_r, mcp, big_cities, market_shed_raster)

12:51:59	1 of 40
12:54:01	2 of 40
12:56:02	3 of 40
12:58:04	4 of 40
13:00:05	5 of 40
13:02:07	6 of 40
13:04:07	7 of 40
13:06:07	8 of 40
13:08:00	9 of 40
13:09:53	10 of 40
13:11:46	11 of 40
13:13:51	12 of 40
13:15:56	13 of 40
13:18:00	14 of 40
13:20:05	15 of 40
13:22:09	16 of 40
13:24:13	17 of 40
13:26:20	18 of 40
13:28:27	19 of 40
13:30:21	20 of 40
13:32:15	21 of 40
13:34:09	22 of 40
13:36:04	23 of 40
13:38:10	24 of 40
13:40:17	25 of 40
13:42:22	26 of 40
13:44:25	27 of 40
13:46:25	28 of 40
13:48:29	29 of 40
13:50:30	30 of 40
13:52:30	31 of 40
13:54:33	32 of 40
13:56:34	33 of 40
13:58:34	34 of 40
14:00:36	35 of 40
14:02:31	36 of 40
14:04:28	37 of 40
14:06:24	38 of 40
14:08:19	39 of 40
14:10:23	40 of 40


In [ ]:
costs = ma.calculate_travel_time(pop_data, traversal_time, uzb_cities, cities_access)

In [ ]:
costs = ma.calculate_travel_time(pop_data, traversal_time, 
                                 big_cities, 
                                 cities_access_50k)

In [ ]:
importlib.reload(ma)
threshold = [30 * 60, 
             60 * 60,
             120 * 60]
features = ma.generate_feature_vectors(network_r, mcp, big_cities, threshold)

In [ ]:
features.crs = {'init':'epsg:4326'}
big_cities['IDX'] = list(range(1, big_cities.shape[0] + 1))
features = pd.merge(features, big_cities.loc[:,['IDX','Name_ENG','y2019']], on='IDX')
features.to_file(os.path.join(os.path.dirname(traversal_raster), "traversal_time_extents.shp"))

In [ ]:
features.drop(['NAME'], axis=1, inplace=True)

# GOSTNets analysis 

In [ ]:
#Extract OSM file
if not os.path.exists(bufferedOSM_pbf):
    uzb_buffered = UZB_b.to_crs(m_proj)
    uzb_buffered['geometry'] = uzb_buffered.buffer(10000)
    uzb_buffered = uzb_buffered.to_crs({'init': 'epsg:4326'})
    globalPbf = '/home/public/Data/GLOBAL/OSM/GLOBAL/planet-latest.osm.pbf'
    osmExtract = osmMisc.osmExtraction(osmosisCmd = "/home/wb411133/Code/Osmosis/bin/osmosis", tempFile = "/home/wb411133/data/temp_osm.sh")
    print(osmExtract.extractBoundingBox(globalPbf, bufferedOSM_pbf, uzb_buffered.unary_union, execute=False))

if not os.path.exists(UZB_cities):
    allCities = gpd.read_file(national_settlements)
    uzb_buffered = UZB_b.to_crs(m_proj)
    uzb_buffered['geometry'] = uzb_buffered.buffer(10000)
    uzb_buffered = uzb_buffered.to_crs(allCities.crs)
    
    uzb_cities = allCities[allCities.intersects(uzb_buffered.unary_union)]
    uzb_cities.to_file(UZB_cities)
else:
    uzb_cities = gpd.read_file(UZB_cities)
    

In [ ]:
network_pickle = os.path.join(outputFolder, "UZB_cleaned_network.pickle")
if not os.path.exists(network_pickle):
    # run GOSTnets access analysis
    G_loader = losm.OSM_to_network(bufferedOSM_pbf)
    G_loader.generateRoadsGDF()
    G = G_loader.initialReadIn()

    #Remove disconnected subgraphs
    largest = 0
    graphs = nx.strongly_connected_component_subgraphs(G)
    for g in graphs:
        if g.number_of_edges() > largest:
            largest = g.number_of_edges()
            selected = g
    G = selected   
    G = gn.convert_network_to_time(G, "length", road_col="infra_type")
    # save processed network to file
    nx.write_gpickle(G, network_pickle)
else:
    G = nx.read_gpickle(network_pickle)

In [ ]:
i_cities = cities.loc[:,['Adm. Center\nENGLISH', 'Adm.Center Population, \n000 people','geometry']]
i_cities.columns = ["Name","Pop","geometry"]
i_cities["Pop"] = i_cities["Pop"] * 1000
i_cities["ISO3"] = "UZB"

In [ ]:
#Create complete cities list
not_uzb_i = uzb_cities['ISO3'] != "UZB"
neighbour_cities = uzb_cities[not_uzb_i]
neighbour_cities = neighbour_cities.loc[:,["Schnm","Pop","ISO3","geometry"]]
neighbour_cities.columns = ["Name","Pop","ISO3","geometry"]
all_cities = i_cities.append(neighbour_cities)

In [ ]:
# Calculate OD
origins = inA.copy()
origins['geometry'] = inA['geometry'].apply(lambda x: x.centroid)
completedOD = gn.Calculate_OD.calculateOD_gdf(G, origins, all_cities)

In [ ]:
uzb_index = all_cities['ISO3'] == "UZB"
not_uzb_i = all_cities['ISO3'] != "UZB"

allGravity= gn.Calculate_OD.calculate_gravity(completedOD)#, dWeight=uzb_cities['Pop'])
uzbOnly   = gn.Calculate_OD.calculate_gravity(completedOD[:,uzb_index])#, dWeight=uzb_cities['Pop'][uzb_index])
notuzb    = gn.Calculate_OD.calculate_gravity(completedOD[:,not_uzb_i])#, dWeight=uzb_cities['Pop'][not_uzb_i])

In [ ]:
origins['allGravity'] = allGravity['d_0.001']
origins['uzbOnlyGravity'] = uzbOnly['d_0.001']
origins['notuzbGravity'] = notuzb['d_0.001']


In [ ]:
origins_gravity = os.path.join(outputFolder, "all_gravity_calculated.shp")
allGravity_file = os.path.join(outputFolder, "all_gravity.csv")
uzbOnly_file = os.path.join(outputFolder,    "uzb_only_gravity.csv")
notuzb_file = os.path.join(outputFolder,     "not_uzb_gravity.csv")

origins.to_file(origins_gravity)
allGravity.to_csv(allGravity_file)
uzbOnly.to_csv(uzbOnly_file)
notuzb.to_csv(notuzb_file)

### gridded analysis

In [ ]:
#J:\Data\GLOBAL\ROAD_NETWORKS\UZB\output
G = nx.read_gpickle("/home/public/Data/GLOBAL/ROAD_NETWORKS/UZB/output/UZB_processed.pickle")
G = gn.convert_network_to_time(G, "length", road_col='infra_type', factor=1000)
# Generate grid for analysis
fishnet = os.path.join(outputFolder, "fishnet_5km.shp")

if not os.path.exists(fishnet):
    b = UZB_b.total_bounds
    misc.createFishnet(fishnet, b[0], b[2], b[1], b[3], 1000, 1000, crsNum=int(UZB_b.crs['init'].split(":")[-1]))
    

In [ ]:
origins = gpd.read_file(fishnet)
dests = cities.loc[:,['Adm. Center\nENGLISH', 'Adm.Center Population, \n000 people','geometry']]
dests.columns = ["Name","Pop","geometry"]
dests["Pop"] = dests["Pop"] * 1000
dests["ISO3"] = "UZB"
#origins = origins.to_crs(dests.crs)

In [ ]:
UZB_b = gpd.read_file(UZB_boundary)
origins['geometry'] = origins['geometry'].apply(lambda x: x.centroid)
select_origins_index = origins.intersects(UZB_b.unary_union)
origins = origins[select_origins_index]
origins = origins.to_crs(dests.crs)

In [ ]:
origins.reset_index(inplace=True)

In [ ]:
completedOD = calcOD.calculateOD_gdf(G, origins, dests, fail_value=999999, calculate_snap=True)

In [ ]:
# Calculate minimum travel time
import numpy as np
minimum_drive_time = np.amin(completedOD, axis=1)

In [ ]:
print(origins.shape)
origins['MinDT'] = minimum_drive_time
# Project UZB boundary to WGS84

print(origins.shape)
origins.to_file(fishnet.replace(".shp", "minimum_drive_time.shp"))

In [ ]:
road_edges = gn.edge_gdf_from_graph(G)
road_edges.drop(['Wkt'], axis=1, inplace=True)
road_edges.to_file(os.path.join(os.path.dirname(fishnet), "UZB_Roads.shp"))

In [ ]:
road_edges.head()

In [ ]:
origins.to_file(fishnet.replace(".shp", "minimum_drive_time.shp"))

# Calculate new urban clusters

In [ ]:
from shapely.wkt import loads

def load_csv_gpd(csv_file, crs={'init':'epsg:4326'}, geom_col = "geometry"):
    ''' convert a csv file to a geopandas dataset.
    
    INPUT
    csv_file [ string ] - path to csv file
    [ optional ] crs [ dictionary ] - CRS dictionary. Default = {'init':'epsg:4326'}
    [ optional ] geom_col [ string ] - column storing geometry. Default = 'geometry'
    
    RETURNS
    [geopandas data frame]
    '''
    inD = pd.read_csv(csv_file, index_col=0)
    inD_geom = inD[geom_col].apply(lambda x: loads(x))
    inD = gpd.GeoDataFrame(inD.drop([geom_col], axis=1), geometry = inD_geom, crs=crs)
    return(inD)

In [ ]:
importlib.reload(GOSTRocks.Urban.UrbanRaster)
wp_urbanExtents = wp_urbanExtents.replace(".csv", "_smoothed.csv")
if not os.path.exists(wp_urbanExtents):
    urban_raster = GOSTRocks.Urban.UrbanRaster.urbanGriddedPop(gridded_pop)
    urban_extents = urban_raster.calculateVectorUrban(densVal=3)
    urban_extents.to_csv(wp_urbanExtents)
    #highDens = urban_raster.calculateVectorUrban(densVal=15, totalPopThresh=50000, smooth=True)
    #highDens.to_csv(wp_densExtents)
else:
    urban_extents = load_csv_gpd(wp_urbanExtents)
    highDens = load_csv_gpd(wp_densExtents)

In [8]:
urban_raster = GOSTRocks.Urban.UrbanRaster.urbanGriddedPop(gridded_pop)
urban_raster.calculateVectorUrban?
    

# Admin level zonal stats

In [ ]:
admin_boundary = "/home/public/Data/COUNTRY/UZB/ADMIN/UZB_adm1.shp"
ghsl_file = r"/home/public/Data/GLOBAL/GHSL/ghsl.vrt"
inA = gpd.read_file(admin_boundary)
inG = rasterio.open(ghsl_file)
inA = inA.to_crs(inG.crs)

In [ ]:
res = rMisc.zonalStats(inA, ghsl_file, rastType='C', unqVals=[1,2,3,4,5,6])

In [ ]:
res_pd = pd.DataFrame(res, columns = ["Water","NotBuilt","Built0014","Built9000","Built7590","BuiltPre75"])
res_pd['NAME'] = inA['NAME_1']
res_pd['TYPE'] = inA['TYPE_1']
res_pd['ID'] = inA['ID_1']
res_pd.to_csv(os.path.join(outputFolder, "ADMIN0_GHSL_Summary.csv"))

In [ ]:
inA.shape

In [ ]:
inA.head()

# City level zonal stats

In [ ]:
inD = gpd.read_file(inputAOI)
viirsOut = os.path.join(outputFolder, "VIIRS_Summaries")
dmspOut  = os.path.join(outputFolder, "DMSP_Summaries")
ghslOut  = os.path.join(outputFolder, "GHSL_Summaries")
popOut   = os.path.join(outputFolder, "Population_Summaries")
for f in [viirsOut, dmspOut, ghslOut, popOut]:
    if not os.path.exists(f):
        os.makedirs(f)


In [ ]:
#Calculate area of urban extents
inD_proj = inD.to_crs(m_proj)
inD['areaKM'] = inD_proj['geometry'].apply(lambda x: x.buffer(0).area/1000000)
inD.drop(['geometry'], axis=1).to_csv(os.path.join(outputFolder, "ExtentArea.csv"))

In [ ]:
# Zonal statistics against viirs
# Get a list of VIIRS vrt files
allVRT = os.listdir(viirsFolder)
for VRT in allVRT:
    print(VRT)
    outFile = os.path.join(viirsOut, VRT.replace(".vrt", ".csv"))
    if not os.path.exists(outFile):
        xx=rMisc.zonalStats(inD, os.path.join(viirsFolder, VRT), minVal=0.1)
        res = pd.DataFrame(xx, columns=['SUM', 'MEAN', 'MAX', 'SD'])
        res['Name'] = inD['Name']
        res.to_csv(outFile)

In [ ]:
# Zonal statistics against DMSP
# get a list of NTL VIIRS
allTif = []
for root, dirs, files in os.walk(dmspFolder):
    for f in files:        
        if f[-24:] == "ElvidgeCorrected_gt3.tif":
            allTif.append(os.path.join(root, f))
            
for dmsp in allTif:
    dmspName = os.path.basename(dmsp)[:7]
    outFile = os.path.join(dmspOut, "%s.csv" % dmspName)
    xx = rMisc.zonalStats(inD, dmsp)
    res = pd.DataFrame(xx, columns=['SUM', 'MEAN', 'MAX', 'SD'])
    res['Name'] = inD['Name']
    res.to_csv(outFile)

In [ ]:
# Zonal statistics against GHSL
xx = rMisc.zonalStats(inD, ghslVRT, rastType='C', reProj=True, unqVals=[0,1,2,3,4,5,6])
res = pd.DataFrame(xx, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6'])
res['Name'] = inD['Name']
res.to_csv(os.path.join(ghslOut, "ghsl.csv"))

In [ ]:
# Zonal statistics against WorldPop
xx = rMisc.zonalStats(inD, gridded_pop, rastType='N', reProj=True, minVal=0)
res = pd.DataFrame(xx, columns=['SUM', 'MEAN', 'MAX', 'SD'])
res['Name'] = inD['Name']
res.to_csv(os.path.join(popOut, "WorldPop.csv"))

In [ ]:
allSummaries = []
for f in os.listdir(viirsOut):
    curRes = rMisc.zonalResult(os.path.join(viirsOut, f), 'N', fieldAction='JOIN', fieldNames = f.replace(".csv",""))
    allSummaries.append(curRes)
    
for f in os.listdir(dmspOut):
    curRes = rMisc.zonalResult(os.path.join(dmspOut, f), 'N', fieldAction='JOIN', fieldNames = f.replace(".csv",""))
    allSummaries.append(curRes)
    
curRes = rMisc.zonalResult(os.path.join(ghslOut, "ghsl.csv"), 'C', fieldAction='JOIN', fieldNames = "GHSL")
allSummaries.append(curRes) 

In [ ]:
allRes = allSummaries[0].inValues
for x in allSummaries[1:]:
    cVals = x.inValues
    cols = [c for c in cVals.columns if not "Name" in c]
    allRes = allRes.join(x.inValues[cols])
allRes = allRes.join(inD)
allRes.to_csv(os.path.join(outputFolder, "allUrbanSummaries.csv"))

# Analyze Urbanization

In [ ]:
# intersect the urban extents with the WP gridded extents
urbanProj = urban_extents.to_crs(m_proj)
inD_proj = inD.to_crs(m_proj)
urbanArea = urbanProj.unary_union
area_res = []
for idx, row in inD_proj.iterrows():
    #get interseting urban areas
    interArea = row['geometry'].intersection(urbanArea)
    area_res.append(interArea.area)

In [ ]:
inD['area_urban_m'] = area_res
inD.drop(['geometry'], axis=1).to_csv(os.path.join(outputFolder, "ExtentArea_Urban.csv"))

# Extract Nighttime Lights

In [ ]:
# Define input and output files
baseline_ntl = os.path.join(viirsFolder, 'VIIRS_201208.vrt')
end_ntl1 = os.path.join(viirsFolder, 'VIIRS_201808.vrt')
end_ntl2 = os.path.join(viirsFolder, 'VIIRS_201904.vrt')
ntl_out_folder = os.path.join(outputFolder, "VIIRS_Data")
if not os.path.exists(ntl_out_folder):
    os.makedirs(ntl_out_folder)
ntlT0_file = os.path.join(ntl_out_folder, "VIIRS_201208.tif")
ntlT1_file = os.path.join(ntl_out_folder, "VIIRS_201808.tif")
ntlT2_file = os.path.join(ntl_out_folder, "VIIRS_201904.tif")
ntl_T1_diff = os.path.join(ntl_out_folder, "VIIRS_diff_201208_201808.tif")
ntl_T2_diff = os.path.join(ntl_out_folder, "VIIRS_diff_201208_201904.tif")

# Project UZB boundary to WGS84
UZB_extent = UZB_b.to_crs({'init':'epsg:4326'})

#Clip T0, T1, T2
rMisc.clipRaster(rasterio.open(baseline_ntl), UZB_extent, ntlT0_file)
rMisc.clipRaster(rasterio.open(end_ntl1), UZB_extent, ntlT1_file)
rMisc.clipRaster(rasterio.open(end_ntl2), UZB_extent, ntlT2_file)

#Write out difference files
ntlT0 = rasterio.open(ntlT0_file)
ntlT1 = rasterio.open(ntlT1_file)
ntlT2 = rasterio.open(ntlT2_file)
if not os.path.exists(ntl_T1_diff):
    with rasterio.open(ntl_T1_diff, 'w', **ntlT0.profile) as dst:
        dst.write(ntlT1.read() - ntlT0.read())        
if not os.path.exists(ntl_T2_diff):
    with rasterio.open(ntl_T2_diff, 'w', **ntlT0.profile) as dst:
        dst.write(ntlT2.read() - ntlT0.read())

# Extract GHSL


In [ ]:
ghsl_vrt = "/home/public/Data/GLOBAL/GHSL/ghsl.vrt"
g_rast = rasterio.open(ghsl_vrt)
ghsl_out_folder = os.path.join(outputFolder, "GHSL_Data")
if not os.path.exists(ghsl_out_folder):
    os.makedirs(ghsl_out_folder)
out_GHSL = os.path.join(ghsl_out_folder, "ghsl.tif")
if not os.path.exists(out_GHSL):
    UZB_extent = UZB_b.to_crs(g_rast.crs)
    rMisc.clipRaster(g_rast, UZB_extent, out_GHSL)


In [ ]:
ntl_T1_diff

## City Scan 

In [ ]:
importlib.reload(osmMisc)
importlib.reload(GOSTRocks.Urban.UrbanAnalysis)
city_scan_folders = '/home/wb411133/data/Country/UZB/city_scans/'
for idx, row in inA.iterrows():
    curCity = row['Name']
    outFolder = os.path.join(city_scan_folders, curCity)
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    inputAOI = os.path.join(outFolder, "AOI.shp")
    gpd.GeoDataFrame([row], crs=inA.crs).to_file(inputAOI)
    gridsize = 250
    #Generate OSM PBF for city
    out_osm_pbf = os.path.join(outFolder, "local.osm.pbf")  
    if not os.path.exists(out_osm_pbf):        
        osmExtract = osmMisc.osmExtraction()
        x = osmExtract.extractBoundingBox(osmPBF, out_osm_pbf, row['geometry'], execute=True)
    else:
        ua = GOSTRocks.Urban.UrbanAnalysis.urbanAnalysis(inShape=inputAOI, 
                     urbanFolder=outFolder, 
                     gridSize=gridsize)
        ua.MarketAccess(osmPBF=out_osm_pbf, churches=False)
        ua.prepMappingData(clipRasters=True)
    print(curCity)
    

In [ ]:
importlib.reload(GOSTRocks.Urban.UrbanAnalysis)
importlib.reload(osmMisc)
ua = GOSTRocks.Urban.UrbanAnalysis.urbanAnalysis(inShape=inputAOI, 
                     urbanFolder=outFolder, 
                     gridSize=gridsize)
ua.MarketAccess(osmPBF=out_osm_pbf, churches=False)
#xx = osmMisc.convertOSMPBF_DataFrame(out_osm_pbf, 'points', ua.inputAOI_WGS84.unary_union)
#xx = ua.osm_pois_file('Health', ['clinic','pharmacy','hospital','health'], out_osm_pbf)